# 데이터 준비 및 나누기

임포트할 거 임포트

In [8]:
# 데이터 읽기 위해 판다스 import
import pandas as pd

train = pd.read_csv("data/train_titanic.csv", index_col = "PassengerId")

# 새로운 칼럼 Sex_encode 만들고 남자 0, 여자 1 주기
train.loc[train["Sex"] == "male", "Sex_encode"] = 0
train.loc[train["Sex"] == "female", "Sex_encode"] = 1
# 확인
train["Sex_encode"]

PassengerId
1      0.0
2      1.0
3      1.0
4      1.0
5      0.0
      ... 
887    0.0
888    1.0
889    1.0
890    0.0
891    0.0
Name: Sex_encode, Length: 891, dtype: float64

In [9]:
# 독립변수와 종속변수 나눔
data = train[["Pclass(F)", "Pclass(B)", "Pclass(E)", "Sex_encode", "Single", "FamilySize"]]
target = train[['Survived']]

# train_test 데이터 나누기 (임포트부터)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size = 0.2, random_state=42)

In [12]:
# 파라미터에 최소불순도 차이를 임의로 지정
# 이걸 할려면 먼저 imort할 거 import
# 의사결정나무 돌리고 그거에 대해 그리드 서치 할 거임
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# (min impurity decrease: 여기에 설정한 값 이상으로 불순도 감소할 경우 노드 분할)

# 그리드 서치를 불러다 놓고 학습도 시킴
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [13]:
# best_estimator_를 사용하여 가장 좋은 성능을 낸 모델 반환
dt = gs.best_estimator_
# 점수도 찍어봄
dt.score(X_train, y_train)

# aha!: .fit이나 .score 다음 () 안에 X_train, y_train 같은 게 들어가는구나 

0.8160112359550562

In [15]:
# best_params_ 통해 가장 좋은 성능 낸 파라미터 찾음

gs.best_params_

{'min_impurity_decrease': 0.0003}

#### 각 매개변수에서 수행한 교차검증 평균 점수 확인

In [16]:
# 확인
gs.cv_results_['mean_test_score']
# .cv_results_ : 파라미터 조합 별 결과조회

array([0.79771496, 0.79771496, 0.79912341, 0.79912341, 0.79912341])

#### 평균 점수에서 가장 큰 값 구하기

In [18]:
# numpy 활용해서 구하네
import numpy as np

# .argmax() 내장함수 활용 -> 변수에 넣어주고, 그리드 서치 내장함수 중 .cv_results_ 활용, 파라미터 값 중 '가장 큰 값' 찾음
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0003}

### 더 복잡한 매개변수 조합 탐색

In [7]:
# 파라미터에 최소불순도차이,max_depth, min_samples_split을 넣어줌
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
         'max_depth': range(5, 20, 1),
         'min_sample_split': range(2, 100, 10)
         }

In [19]:
# gs에 또 학습시킴
gs = GridSearchCV(DecisionTreeClassifier(random_state=42, params, n_jobs=-1))
gs.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [20]:
# 가장 좋은 파라미터 조합 찾음
gs.best_params_

{'min_impurity_decrease': 0.0003}

In [21]:
# 교차검증 평균 점수 찾기
np.max(gs.cv_results_['mean_test_score'])

0.7991234117994681

In [23]:
# scipy에서 uniform, randint 임포트
# 0부터 10까지 랜덤한 정수 고르고 하나의 변수에 넣어줌
from scipy.stats import uniform, randint
rgen = randint(0,10)

# .rvs()로 무작위 표본 생성
rgen.rvs(10)

array([9, 8, 9, 4, 5, 7, 3, 6, 7, 1])

In [5]:
# 샘플링 갯수를 늘리고 중복값 제거, 고르게 가져오는 거 확인
np.unique(rgen.rvs(1000), return_counts = True)
# rvs: Random Value Sampling
# return_counts = True: 값들의 빈도가 담긴 array를 리턴

In [6]:
# 0 ~ 1 사이의 10개 실수를 무작위 추출
# uniform() 함수 활용
ugen = uniform(0,1)
ugen.rvs(10)

In [38]:
# min_samples_leaf와 RandomizedSearchCV 사용해서 가장 좋은 결과 낸 파라미터 조합 조회
# 하지만 모든 조합을 다 시도하지는 않고, 각 반복마다 임의의 값만 대입해 지정한 횟수만큼 평가
# 위에 지정한 세 가지 파라미터에 min_samples_leaf 까지 추가
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
         'max_depth': randint(20, 50),
         'min_samples_split': randint(2, 25),
         'min_samples_leaf': randint(1,25)
         }

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
# n_iter: for loop 몇 번 도는지 세어줌

# 모델 학습시킴
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E46D4666D0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E46D518A60>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E46D466670>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E46D518520>},
                   random_state=42)

In [26]:
# 가장 좋은 파라미터 조합 확인
gs.best_params_

{'min_impurity_decrease': 0.0003}

In [27]:
# 교차 검증 평균 점수 확인
np.max(gs.cv_results_['mean_test_score'])

0.7991234117994681

### 테스트 데이터셋 점수 확인

In [39]:
dt = gs.best_estimator_

dt.score(X_test, y_test)

0.7988826815642458